In [28]:
from dotenv import load_dotenv
load_dotenv()
import os

os.environ["LITELLM_LOG"] = "CRITICAL"
os.environ["LITELLM_DISABLE_LOGGING"] = "true"
os.environ["LITELLM_LOG_LEVEL"] = "CRITICAL"
# Retry config
# os.environ["LITELLM_MAX_RETRIES"] = "3"
# os.environ["LITELLM_RETRY_DELAY"] = "2"   # seconds
# os.environ["LITELLM_TIMEOUT"] = "60"

In [38]:
original_email = """looping in priya. TAX and PRS updtes are in the deck."""
# Lets sync up tommorow if SYNCBOT allows :). Ping me if any blockers.
# ETA for SDS Integration if Friday

from crewai.tools import BaseTool

class ReplaceJargonsTool(BaseTool):
    name : str = "rewrite_email"
    description : str = "Replace Jargons with more specific terms"

    def _run(self, email:str) -> str :
        replacements = {
            "PRS" : "Project Phoenix (Internal AI Revamp project)",
            "TAX" : "Technial Architecture Stack",
            "DBS" : "Client Database cluster",
            "SDS" : "Smart Data Syncher",
            "SYNCBOT" : "Internal standup assistant bot",
            "WIP" : "In Progress",
            "POC" : "Proof of Concept",
            "ping" : "Reach out"
        }
        suggestions = []
        email_lower = email.lower()
        for jargon,replacement in replacements.items():
            if jargon.lower() in email_lower:
                suggestions.append(f"Consider replacing '{jargon}' with '{replacement}'")


        return "\n".join(suggestions) if suggestions else "No jargon or internal abbreviations replaced"

jt = ReplaceJargonsTool()
jt.run(original_email)

Using Tool: rewrite_email


"Consider replacing 'PRS' with 'Project Phoenix (Internal AI Revamp project)'\nConsider replacing 'TAX' with 'Technial Architecture Stack'\nConsider replacing 'ping' with 'Reach out'"

In [39]:
from crewai import LLM
import os

llm  = LLM(model="groq/llama-3.1-8b-instant",temperature=0, provider="groq")
#llm  = LLM(model="gemini/gemini-2.0-flash",temperature=0)

llm.call("Who invented transcendental meditation?")

'Transcendental meditation (TM) was developed by Maharishi Mahesh Yogi. He was an Indian guru who founded the Transcendental Meditation movement in the 1950s. Maharishi Mahesh Yogi was born on January 12, 1917, in Jabalpur, India, and died on February 5, 2008, in Vlodrop, Netherlands.\n\nMaharishi Mahesh Yogi was a student of Swami Brahmananda Saraswati, a Hindu guru, and he learned the technique of transcendental meditation from him. Maharishi Mahesh Yogi later developed his own version of the technique, which he claimed was a more efficient and effective way to achieve a state of consciousness known as "pure consciousness."\n\nMaharishi Mahesh Yogi popularized TM through his teachings and writings, and it gained a significant following in the 1960s and 1970s, particularly among celebrities and intellectuals. However, the movement has also been the subject of controversy and criticism over the years, with some accusing it of being a cult or a money-making scheme.\n\nDespite these crit

In [40]:
from crewai import Agent, Task, Crew

email_assistant = Agent(
    role = "Email Assistant Agent",
    goal = "Rewrite the email using the replace_jargons tool and return only the rewritten email",
    backstory = "You simplify business emails and do not add commentary.",
    tools =[jt],
    verbose = True,
    llm = llm
)



email_task = Task(
    description = f""" Take the following rough email and rewrite into a professional and polished version.
    Expand abbreviations using Replace_Jargons tool:
    '''{original_email}'''""",
    agent = email_assistant,
    expected_output="A professional written email with proper formatting and content",
)

crew = Crew(
    agents = [email_assistant],
    tasks=[email_task],
    verbose = True,
)

# from litellm.exceptions import RateLimitError
# import time

# for attempt in range(3):
#     try:
#         result = crew.kickoff()
#         print(result)
#         break
#     except RateLimitError:
#         time.sleep(6)
# else:
#     raise RuntimeError("Groq rate limit retry failed")
result = crew.kickoff()
print(result)

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 8840b244-7a06-4697-b5b1-61dc79eb32aa                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Email Assistant Agent                                                                                   │
│                                                                                                                 │
│  Task:  Take the following rough email and rewrite into a professional and polished version.                    │
│      Expand abbreviations using Replace_Jargons tool:                                                           │
│      '''looping in priya. TAX and PRS updtes are in the deck.'''                                                │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────── 🔧 Agent Tool Execution ────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Email Assistant Agent                                                                                   │
│                                                                                                                 │
│  Thought: Action: rewrite_email                                                                                 │
│                                                                                                                 │
│  Using Tool: rewrite_email                                                                                      │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── Tool Input ───────────────────────────────────────────────────╮
│                                                                                                                 │
│  {                                                                                                              │
│    "email": "Expand abbreviations using Replace_Jargons tool: looping in priya. TAX and PRS updtes are in the   │
│  deck."                                                                                                         │
│  }                                                                                                              │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── Tool Output ──────────────────────────────────────────────────╮
│                                                                                                                 │
│  Consider replacing 'PRS' with 'Project Phoenix (Internal AI Revamp project)'                                   │
│  Consider replacing 'TAX' with 'Technial Architecture Stack'                                                    │
│  Consider replacing 'ping' with 'Reach out'                                                                     │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

...

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Email Assistant Agent                                                                                   │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  Subject: Update on Technical Architecture Stack and Project Phoenix (Internal AI Revamp project)               │
│                                                                                                                 │
│  Dear Priya,                                                                                                    │
│                                                                                                                 │
│  I wanted to reach out and let you know that the updates on the Technical Architecture Stack and Project        │
│  Phoenix (Internal AI Revamp project) are now available in the deck.                                            │
│                                                                                                                 │
│  Best regards,                                                                                                  │
│  [Your Name]                                                                                                    │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Subject: Update on Technical Architecture Stack and Project Phoenix (Internal AI Revamp project)

Dear Priya,

I wanted to reach out and let you know that the updates on the Technical Architecture Stack and Project Phoenix (Internal AI Revamp project) are now available in the deck.

Best regards,
[Your Name]


╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 8840b244-7a06-4697-b5b1-61dc79eb32aa                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output: Subject: Update on Technical Architecture Stack and Project Phoenix (Internal AI Revamp          │
│  project)                                                                                                       │
│                                                                                                                 │
│  Dear Priya,                                                                                                    │
│                                                                                                                 │
│  I wanted to reach out and let you know that the updates on the Technical Architecture Stack and Project        │
│  Phoenix (Internal AI Revamp project) are now available in the deck.                                            │
│                                                                                                                 │
│  Best regards,                                                                                                  │
│  [Your Name]                                                                                                    │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Tracing Status ─────────────────────────────────────────────────╮
│                                                                                                                 │
│  Info: Tracing is disabled.                                                                                     │
│                                                                                                                 │
│  To enable tracing, do any one of these:                                                                        │
│  • Set tracing=True in your Crew/Flow code                                                                      │
│  • Set CREWAI_TRACING_ENABLED=true in your project's .env file                                                  │
│  • Run: crewai traces enable                                                                                    │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: d1f5e833-1b84-493c-887c-5c994e8be92c                                                                     │
│  Agent: Email Assistant Agent                                                                                   │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯